# HW2 — Bike Sharing Demand Prediction (day.csv)

**Follow CRISP-DM**: Business Understanding, Data Understanding, Data Preparation, Modeling, Evaluation, Deployment.

This notebook uses `day.csv` (Kaggle Bike Sharing). **Before running**, set `DATA_PATH` to the full path of your local `day.csv` file (example provided).

## Setup
Install required packages if needed (run once):
```bash
pip install -U scikit-learn pandas matplotlib nbformat scipy
```

In [ ]:
# === 0. Set dataset path ===
# UPDATE this path to where your day.csv file is located on your machine or environment.
DATA_PATH = '/root/.cache/kagglehub/datasets/gauravduttakiit/bike-sharing/versions/1/day.csv'

# If your file is in the current working directory, you can use:
# DATA_PATH = 'day.csv'

print('Make sure DATA_PATH points to your day.csv file:')
print(DATA_PATH)

## 1. Business Understanding
Predict daily bike rentals (`cnt`) to support operations and resource allocation.

Deliverables required by assignment:
- Use a Kaggle dataset (10–20 features) and cite the link.
- Use Linear Regression; perform Feature Selection & Model Evaluation.
- Include prediction plot with confidence/prediction interval.
- Provide CRISP-DM documentation in report.

## 2. Data Understanding
Load dataset and inspect.

In [ ]:
import pandas as pd

# Load dataset
try:
    df = pd.read_csv(DATA_PATH)
    print('Loaded dataset shape:', df.shape)
    display(df.head())
except Exception as e:
    print('Error loading dataset. Please check DATA_PATH. Error:', e)

## 3. Data Preparation
- Drop leaking or unnecessary columns: `instant`, `dteday`, `casual`, `registered`.
- Identify categorical vs numeric features.
- Preprocessing: StandardScaler (numeric) + OneHotEncoder (categorical).

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Prepare X, y
cols_to_drop = ['instant','dteday','casual','registered','cnt']
X = df.drop(columns=[c for c in cols_to_drop if c in df.columns])
y = df['cnt']

categorical_features = [c for c in ['season','yr','mnth','holiday','weekday','workingday','weathersit'] if c in X.columns]
numeric_features = [c for c in X.columns if c not in categorical_features]

print('Numeric features:', numeric_features)
print('Categorical features:', categorical_features)

preprocessor = ColumnTransformer(transformers=[
    ('num', StandardScaler(), numeric_features),
    ('cat', OneHotEncoder(drop='first', sparse=False), categorical_features)
])

# Fit preprocessor to get transformed feature names
X_trans = preprocessor.fit_transform(X)
cat_encoder = preprocessor.named_transformers_['cat']
cat_feature_names = list(cat_encoder.get_feature_names_out(categorical_features)) if categorical_features else []
feature_names = numeric_features + cat_feature_names
print('\nTotal features after preprocessing:', len(feature_names))
print(feature_names[:30])

## 4. Feature Selection (RFE)
We'll use RFE with LinearRegression to select top 10 features (or fewer if fewer exist).

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE

# Transform X to array for RFE
X_array = preprocessor.transform(X)

n_total = X_array.shape[1]
n_select = min(10, n_total)

rfe = RFE(estimator=LinearRegression(), n_features_to_select=n_select)
rfe.fit(X_array, y)

# Selected feature names
selected_mask = rfe.support_
selected_features = [fname for fname, sel in zip(feature_names, selected_mask) if sel]

import pandas as pd
ranking_df = pd.DataFrame({'feature': feature_names, 'selected': selected_mask, 'ranking': rfe.ranking_})
print('Selected features (RFE):', selected_features)
display(ranking_df.sort_values(['selected','ranking']).head(20))

## 5. Modeling
Train a LinearRegression model using only the selected features. We'll split data into train/test (80/20).

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Build selector transformer
class ColumnSelector:
    def __init__(self, mask):
        self.mask = np.array(mask)
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[:, self.mask]

selector = ColumnSelector(selected_mask)

final_pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('selector', selector), ('lr', LinearRegression())])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
final_pipeline.fit(X_train, y_train)

y_pred = final_pipeline.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
print(f'RMSE: {rmse:.2f}')
print(f'R2: {r2:.4f}')

## 6. Evaluation
- Metrics: RMSE, R²
- Top 5 outliers (largest absolute residuals)
- Plots: Actual vs Predicted + 95% prediction intervals

In [ ]:
# Residuals and top outliers
residuals = pd.Series(y_test - y_pred, index=y_test.index)
abs_res = residuals.abs().sort_values(ascending=False)
top5 = abs_res.head(5)
print('Top 5 outliers (index : abs residual):')
print(top5)

# Prepare prediction intervals using linear model theory
# Reconstruct design matrix for selected features
X_train_trans = preprocessor.transform(X_train)
X_train_sel = X_train_trans[:, selected_mask]
X_train_design = np.hstack([np.ones((X_train_sel.shape[0],1)), X_train_sel])
XtX_inv = np.linalg.inv(X_train_design.T.dot(X_train_design))

X_test_trans = preprocessor.transform(X_test)
X_test_sel = X_test_trans[:, selected_mask]
X_test_design = np.hstack([np.ones((X_test_sel.shape[0],1)), X_test_sel])

n_train = X_train_design.shape[0]
p = X_train_design.shape[1]

rss = np.sum((y_train - final_pipeline.predict(X_train))**2)
sigma_hat = np.sqrt(rss / (n_train - p))

from scipy.stats import t
tval = t.ppf(0.975, df=n_train - p)
se_pred = np.array([sigma_hat * np.sqrt(1.0 + x0.reshape(1,-1).dot(XtX_inv).dot(x0.reshape(-1,1))[0,0]) for x0 in X_test_design])
lower = y_pred - tval * se_pred
upper = y_pred + tval * se_pred

print('\nPrediction intervals computed (95%).')

In [ ]:
# Plot Actual vs Predicted with prediction intervals
import matplotlib.pyplot as plt

# Scatter Actual vs Predicted
plt.figure(figsize=(8,6))
plt.scatter(y_test, y_pred)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], linestyle='--')
plt.xlabel('Actual count')
plt.ylabel('Predicted count')
plt.title('Actual vs Predicted (Linear Regression)')
plt.tight_layout()
plt.show()

# Sorted plot with intervals
order = np.argsort(y_test.values)
plt.figure(figsize=(10,6))
plt.plot(range(len(y_test)), y_test.values[order], label='Actual')
plt.plot(range(len(y_test)), y_pred[order], label='Predicted')
plt.fill_between(range(len(y_test)), lower[order].flatten(), upper[order].flatten(), alpha=0.25)
plt.legend()
plt.title('Actual and Predicted with 95% Prediction Intervals (sorted by actual)')
plt.xlabel('Sorted test samples')
plt.ylabel('Count')
plt.tight_layout()
plt.show()

## 7. Deployment
- Save the model with `joblib.dump()` if needed.
- The notebook and results (figures, CSVs) can be exported to include in the PDF report.

---

## Notes for the report (to include in PDF):
- **Dataset source & link:** add Kaggle URL and dataset version used.
- **GPT 輔助內容:** paste transcript exported via pdfCrowd.
- **NotebookLM 摘要:** include a 100+ word summary of external research.

---

**End of notebook.**